In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from skbio.stats.distance import permanova
from skbio.stats.distance import DistanceMatrix
from scipy import stats
from scikit_posthocs import posthoc_dunn as dunn
from statsmodels.stats.multitest import multipletests
from itertools import combinations

%matplotlib inline

!mkdir Results/Figures

mkdir: cannot create directory ‘Results/Figures’: File exists


In [ ]:
!pip install scikit-posthocs
!pip install scikit-bio

# Alpha diversity

In [4]:
import pandas as pd
def alpha_div(qza):  
  a = !unzip $qza
  digest = a[1].split('/')[0].replace('  inflating: ','')
  inf = digest + '/data/alpha-diversity.tsv'
  data = pd.read_csv(inf, sep='\t',index_col=0)
  !rm -r $digest
  return data 

alpha = pd.read_csv('metadata.tsv', sep='\t', index_col='#SampleID')
diversity = {'observed_features':'Observed ASVs',
             'shannon':'Shannon\'s entropy',
             'faith_pd':'Faith\'s PD',
             'evenness':'Pielou\'s evenness'}

for div in diversity:
  vector = 'Results/Core-metrics/%s_vector.qza' % div
  metric = div
  if div == 'shannon':
    metric = 'shannon_entropy'
  if div == 'evenness':
    metric = 'pielou_evenness'
  alpha_df = alpha_div(vector)
  alpha = pd.concat([alpha, alpha_df[metric]], axis=1)
alpha.index.name = '#SampleID'
alpha = alpha[alpha['faith_pd'].notna()]
alpha['Day_Trt'] = alpha.Day+'_'+alpha.Treatment
alpha.to_csv('Results/Core-metrics/alpha.tsv',sep='\t')

def kw_dunn(df,col,metric,pairwise):
  phoc = pd.DataFrame(columns=['Group1','Group2','p','q'])
  kw = stats.kruskal(*[g[metric].values for n,g in df.groupby(col)])
  if kw[1] < 0.05 and pairwise==True:
    dunn_p = dunn(df,val_col=metric,group_col=col)
    dunn_q = dunn(df,val_col=metric,group_col=col,p_adjust='fdr_bh')
    for i,pair in enumerate(combinations(set(df[col].tolist()),2)):
      phoc.loc[len(phoc)]=[pair[0],pair[1],dunn_p.loc[pair[0],pair[1]],dunn_q.loc[pair[0],pair[1]]]
  return kw,phoc

def color_df(df):
  dfcol = df.style.applymap(lambda x: "background-color: red" if x<0.05 \
  else "background-color: grey",subset=pd.IndexSlice[:, [c for c in df.columns if c in 'pq']])
  return dfcol

diversity = {'observed_features':'Observed ASVs','shannon_entropy':'Shannon\'s entropy',
             'faith_pd':'Faith\'s PD','pielou_evenness':'Pielou\'s evenness'}
alpha = pd.read_csv('Results/Core-metrics/alpha.tsv',sep='\t',index_col='#SampleID')

# Beta diversity

In [19]:
!conda install -y -c bioconda git
!git clone https://github.com/Auerilas/ecopy.git ecopy_source
!pip install ecopy_source/
!rm -f ecopy_source

In [2]:
import numpy as np
import pandas as pd
import ecopy as ep

def beta_div(qza):  
  a = !unzip $qza
  digest = a[1].split('/')[0].replace('  inflating: ','')
  inf = digest + '/data/distance-matrix.tsv'
  data = pd.read_csv(inf, sep='\t',index_col=0)
  !rm -r $digest
  return data 

def ecopy_nmds(dist,col):
  sns.set_style("ticks")
  !mkdir Results/Biplots/Filtered
  MDS = ep.MDS(dist,naxes=2,transform='monotone')
  stress=round(MDS.stress,3)
  scores_df=pd.DataFrame(MDS.scores,index=dist.columns)
  scores_df.columns = ['NMDS1','NMDS2']
  met = pd.read_csv('metadata.tsv', sep='\t', index_col='#SampleID')
  scores_df['BS']=[met.loc[n,'BS'] for n in dist.index]
  scores_df['Day']=[met.loc[n,'Day_num'] for n in dist.index]
  scores_df['Trt']=[met.loc[n,'Treatment'] for n in dist.index]
  scores_df['HP']=[met.loc[n,'Source'] for n in dist.index]
  mdict = {'E':'^', 'FL':'o', 'FR':'X', 'RF':'d', 'RSP':'s', 'SAM':'*'}
  hue,s = 'Trt','Day'
  cdict = {'Trt1':'grey','Trt2':'yellow','Trt3':'red','Trt4':'orange',\
           'Trt5':'green','Trt6':'blue','not_appl':'brown'}
  if 'Src' in col:
    hue = 'HP'
    cdict = {'HP1':'red','HP2':'blue','not_appl':'white'}   
  elif 'Dh' in col or 'Day_hour' in col: 
    s = 'Hour'
    scores_df['Hour']=[int(met.loc[n,'Day_hour'][-2:]) for n in dist.index]
  fig,ax=plt.subplots(figsize=(2.5,2.5),dpi=300)
  sns.scatterplot(x='NMDS1',y='NMDS2',data=scores_df,ax=ax,size=s,style='BS',\
                  markers=mdict,hue=hue,palette=cdict)
  ax.legend(bbox_to_anchor=(1,1),loc=2,fontsize=6,title=None,frameon=False,markerscale=0.7,\
           borderpad=0,handletextpad=0.1)
  ax.tick_params(axis='both', length=0, pad=0)       
  ax.set_xticklabels([])
  ax.set_yticklabels([])
  ax.set_ylabel('NMDS2',fontdict={'fontsize': 6})
  ax.set_xlabel('NMDS1',fontdict={'fontsize': 6})
  ax.text(0.01,0.01,'Stress '+str(stress),size=4,transform=ax.transAxes)
  
def to_matrix(df1,df2):
  df1 = df1[df1.index.isin(df2.index)]
  df1 = df1[df2.index]
  df1.sort_index(inplace=True)
  df1 = df1.reindex(sorted(df1.columns),axis=1)
  matrix = DistanceMatrix(df1,ids=df1.index)
  return matrix, df1

def permanova_pairwise(df,col,met_d,pairwise):
  matrix, df1 = to_matrix(df.copy(),met_d)
  perm = permanova(matrix,met_d,column=col,permutations=999)
  perm_pair = pd.DataFrame(columns=['Group1','Group2','p'])
  if perm[5] < 0.05 and pairwise==True:
    for pair in combinations(set(met_d[col].tolist()),2):
      metapairs = met_d.loc[(met_d[col]==pair[0])|(met_d[col]==pair[1])].copy()
      matrix2 = to_matrix(df.copy(),metapairs)[0]
      pp = permanova(matrix2,metapairs,column=col,permutations=999)[5]
      perm_pair.loc[len(perm_pair)]=[pair[0],pair[1],pp]
    perm_pair['q'] = multipletests(perm_pair.p,method='fdr_bh')[1]      
  return perm, perm_pair, df1

def mk_biplot(distances,group,div):
  distances.to_csv('Results/Biplots/Filtered/%s_%s_distances.tsv'%(div,group),sep='\t')
  dist = 'Results/Biplots/Filtered/%s_%s_distances.tsv'%(div,group)
  matr = 'Results/Biplots/Filtered/%s_%s_distances.qza'%(div,group)
  pcoa = 'Results/Biplots/Filtered/%s_%s_pcoa.qza'     %(div,group)
  relt = 'Data/Relative_tables/%s_%s_rel_table.qza'    %(div,group)
  bipl = 'Results/Biplots/Filtered/%s_%s_biplot.qza'   %(div,group)
  bipv = 'Results/Biplots/Filtered/%s_%s_biplot.qzv'   %(div,group)
          
  !qiime tools import \
    --input-path $dist \
    --output-path $matr \
    --type DistanceMatrix
  !qiime diversity pcoa \
    --i-distance-matrix $matr \
    --o-pcoa $pcoa
          
  distances.index.name = '#SampleID'
  distances.to_csv('Results/Biplots/Filtered/%s_%s_distances.tsv'%(div,group),sep='\t')
          
  !qiime feature-table filter-samples \
    --i-table Data/Relative_tables/full-relative_table.qza \
    --m-metadata-file $dist \
    --o-filtered-table $relt
  !qiime diversity pcoa-biplot \
    --i-pcoa $pcoa \
    --i-features $relt \
    --o-biplot $bipl
  !qiime emperor biplot \
    --i-biplot $bipl \
    --m-sample-metadata-file metadata.tsv \
    --p-number-of-features 5 \
    --o-visualization $bipv
  
def color_df(df):
  dfcol = df.style.applymap(lambda x: "background-color: red" if x<=0.05 \
  else "background-color: grey",subset=pd.IndexSlice[:, [c for c in df.columns if c in 'pq']])
  return dfcol

def tovector(df,col,meta):
  df = df.stack().reset_index().rename(columns={'level_0':'Sample1','level_1':'Sample2', 0:'Distance'}).copy()
  for i in meta.index:
    df.loc[df.Sample1==i,'Group1'] = meta.loc[i,col]
    df.loc[df.Sample2==i,'Group2'] = meta.loc[i,col]
  return(df)

meta = pd.read_csv('Results/Core-metrics/alpha.tsv', sep='\t', index_col='#SampleID')
diversity = {'jaccard':'Jaccard','bray_curtis':'Bray-Curtis'}
alpha = pd.read_csv('Results/Core-metrics/alpha.tsv',sep='\t',index_col='#SampleID')
!mkdir Results/Biplots/Filtered

mkdir: cannot create directory ‘Results/Biplots/Filtered’: File exists


### Permanova overall test

In [ ]:
cols = ['BS','Day_hour','Treatment','Source','rstc_run']
!mkdir Results/Beta_comp
meta = meta.sort_values(['Day_num','BS'],ascending=[True,True])
groups = meta.Day.unique().tolist()+meta.BS_Day.unique().tolist()
summary,i = pd.DataFrame(),0

def summarize():
  summary.loc[i,'Metric'] = div
  summary.loc[i,'Group'] = g
  summary.loc[i,'Column'] = col
  summary.loc[i,'Perm.'] = str(perm[6])
  summary.loc[i,'PERMstats'] = round(perm[4],5)
  summary.loc[i,'p'] = round(perm[5],5)
  if perm[5] > 0.05:
    summary.loc[i,'Comment'] = 'no_pairwise'
  if perm[5] <= 0.1:
    summary.loc[i,'Comment'] = 'trend'
  if perm[5] <= 0.05:
    summary.loc[i,'Comment'] = 'pairwise'
  if perm[5] <= 0.05 and len(met_d[col].unique()) == 2:
    summary.loc[i,'Comment'] = 'p = %s'%round(perm[5],5)
  
for div in diversity:
  data = beta_div('Results/Core-metrics/%s_distance_matrix.qza'%div)
  col,g,met_d = 'Day','All',meta.copy()
  perm,perm_pair,distances = permanova_pairwise(data.copy(),col,meta,pairwise=False)
  summarize()
  i += 1
  for g in groups:
    for col in cols:
      met_d = meta.loc[((meta.BS_Day==g)&(~meta[col].str.contains('not_appl')))].copy()
      if col == 'BS' and g in alpha.Day.unique():
        met_d = meta.loc[meta.Day==g].copy()
      if len(met_d[col].unique()) < 2: continue
      perm,perm_pair,distances = permanova_pairwise(data.copy(),col,met_d,pairwise=False)
      summarize()
      i += 1
summary.to_csv('Results/Beta_comp/Summary_Permanova_all.tsv',sep='\t')
display(color_df(summary))

### Pairwise differences between sample-types by days

In [ ]:
meta = meta.sort_values(['Day_num','BS'],ascending=[True,True])
groups = meta.Day.unique().tolist()

for bet in diversity:
  beta = beta_div('Results/Core-metrics/%s_distance_matrix.qza'%bet)
  for g in groups:
    data = meta.loc[(meta.Day==g)].copy()
    perm,perm_pair,distances = permanova_pairwise(beta,'BS',data,pairwise=True)
    perm.to_csv('Results/Beta_comp/Perm_%s-%s_BSbyDays.tsv'%(g,bet),sep='\t')
    perm_pair.to_csv('Results/Beta_comp/Perm_pairs_%s-%s_BSbyDays.tsv'%(g,bet),sep='\t')
    print(g,bet,'******************************************')
    display(perm)
    if perm[5] <= 0.05:
      display(color_df(perm_pair))

In [3]:
groups = meta.BS_Day.unique().tolist()

for bet in diversity:
  beta = beta_div('Results/Core-metrics/%s_distance_matrix.qza'%bet)
  for g in groups:
    data = meta.loc[(meta.BS_Day==g)].copy()
    if 'd0' in  g: continue
    perm,perm_pair,distances = permanova_pairwise(beta,'Treatment',data,pairwise=True)
    mk_biplot(distances,g,bet)
    
    dist = 'Results/Biplots/Filtered/%s_%s_distances.qza'%(bet,g)
    vizn = 'Results/Beta_comp/%s_%s_Trt_permanova.qzv'%(bet,g)
    !qiime diversity beta-group-significance \
      --i-distance-matrix $dist \
      --m-metadata-file metadata.tsv \
      --m-metadata-column Treatment \
      --p-method 'permanova' \
      --p-pairwise \
      --o-visualization $vizn
    
    if g == 'FR_d13':
      vizn = 'Results/Beta_comp/%s_%s_Hours_permanova.qzv'%(bet,g)
      !qiime diversity beta-group-significance \
        --i-distance-matrix $dist \
        --m-metadata-file metadata.tsv \
        --m-metadata-column Day_hour \
        --p-method 'permanova' \
        --p-pairwise \
        --o-visualization $vizn

Imported Results/Biplots/Filtered/jaccard_FL_d13_distances.tsv as DistanceMatrixDirectoryFormat to Results/Biplots/Filtered/jaccard_FL_d13_distances.qza
Saved PCoAResults to: Results/Biplots/Filtered/jaccard_FL_d13_pcoa.qza
Saved FeatureTable[RelativeFrequency] to: Data/Relative_tables/jaccard_FL_d13_rel_table.qza
Saved PCoAResults % Properties('biplot') to: Results/Biplots/Filtered/jaccard_FL_d13_biplot.qza
Saved Visualization to: Results/Biplots/Filtered/jaccard_FL_d13_biplot.qzv
Saved Visualization to: Results/Beta_comp/jaccard_FL_d13_Trt_permanova.qzv
Imported Results/Biplots/Filtered/jaccard_E_d7_distances.tsv as DistanceMatrixDirectoryFormat to Results/Biplots/Filtered/jaccard_E_d7_distances.qza
Saved PCoAResults to: Results/Biplots/Filtered/jaccard_E_d7_pcoa.qza
Saved FeatureTable[RelativeFrequency] to: Data/Relative_tables/jaccard_E_d7_rel_table.qza
Saved PCoAResults % Properties('biplot') to: Results/Biplots/Filtered/jaccard_E_d7_biplot.qza
Saved Visualization to: Results/Bipl

### Sample types within treatments

In [ ]:
meta = meta.sort_values(['Day_num','BS'],ascending=[True,True])
groups = meta.Day_Trt.unique().tolist()

for bet in diversity:
  beta = beta_div('Results/Core-metrics/%s_distance_matrix.qza'%bet)
  for g in groups:
    data = meta.loc[(meta.Day_Trt==g)].copy()
    if g in groups[:2]: 
      continue
    perm,perm_pair,distances = permanova_pairwise(beta,'BS',data,pairwise=True)
    perm.to_csv('Results/Beta_comp/Perm_%s-%s_byTrt.tsv'%(g,bet),sep='\t')
    perm_pair.to_csv('Results/Beta_comp/Perm_pairs_%s-%s_byTrt.tsv'%(g,bet),sep='\t')
    print(g,bet,'******************************************')
    display(perm)
    if perm[5] <= 0.05:
      display(color_df(perm_pair))